# Bayesian Estimation of Marble League 2020 Standings
---
$$
\gdef\P#1 {{ \mathrm{P} \! \left( #1 \right)}}
$$

We want to find:

- The distribution of the final champion team, $\P {\text{Champion} = i}$, of the 2020 Marble League
- Or more generally, the distribution per final ranking $\P {\text{Ranking}, \text{Team}}$ (e.g. $\P {\text{Champion} = i} = \P {\text{Ranking} = 1, \text{Team} = i}$)
- The latent "strength" of each team, $\P {\text{Strength}}$, which is a random variable because the performance is different in every event.

The Model:

![](Model.svg)

where:
- $\text{Event Performance} \sim \mathrm{Beta} \left( a, b \right)$

In [50]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pymc3 as pm
import scipy.stats as stats
# import theano.tensor as tt
import ipywidgets as widgets
plt.style.use('ggplot')

In [9]:
standings = pd.read_csv('standings.csv', index_col=0)
standings

,Balls of Chaos,Bumblebees,Crazy Cat's Eyes,Green Ducks,Hazers,Hornets,Mellow Yellow,Midnight Wisps,Minty Maniacs,O'rangers,Oceanics,Raspberry Racers,Savage Speeders,Team Galactic,Team Momo,Thunderbolts
E1,4,8,5,9,7,12,16,3,1,10,10,2,14,15,6,13
E2,13,10,2,12,8,7,11,4,3,1,16,9,15,5,6,14
E3,5,7,6,4,10,12,15,11,1,2,13,9,3,14,16,8
E4,13,8,4,10,7,15,16,5,6,1,12,3,9,2,11,14
E5,2,14,1,13,10,16,3,11,8,12,6,15,4,7,8,5
E6,15,10,4,16,9,7,14,2,8,6,13,1,3,5,12,11
E7,13,15,7,9,16,8,10,1,14,3,12,6,4,5,2,11
E8,13,14,12,15,1,4,16,6,10,3,9,11,7,5,2,7
E9,15,8,7,16,2,13,5,10,3,12,1,11,4,9,6,14
E10,13,9,12,6,5,16,15,11,1,10,2,4,3,7,14,8


In [10]:
teams = standings.columns
teams

Index(['Balls of Chaos', 'Bumblebees', 'Crazy Cat's Eyes', 'Green Ducks',
       'Hazers', 'Hornets', 'Mellow Yellow', 'Midnight Wisps', 'Minty Maniacs',
       'O'rangers', 'Oceanics', 'Raspberry Racers', 'Savage Speeders',
       'Team Galactic', 'Team Momo', 'Thunderbolts'],
      dtype='object')

In [16]:
# Let's only consider the two teams O'rangers and Savage Speeders
orangers_vs_speeders = standings[['O\'rangers', 'Savage Speeders']]
orangers_vs_speeders = orangers_vs_speeders.rank(axis=1).astype(np.int)
orangers_vs_speeders = -(orangers_vs_speeders - 2)
orangers_vs_speeders

,O'rangers,Savage Speeders
E1,1,0
E2,1,0
E3,1,0
E4,1,0
E5,0,1
E6,0,1
E7,1,0
E8,1,0
E9,0,1
E10,0,1


In [54]:
# Let's calculate the posterior of the relative strength of O'rangers over
# Savage Speeders
def likelihood(p, D):
    ones = (D == 1).sum()
    zeros = (D == 0).sum()
    return (p ** ones) * ((1 - p) ** zeros)

def plot(N, a):
    prior = lambda p: stats.beta(a=a, b=a).pdf(p)
    x = np.linspace(0, 1, 1000)
    pri = prior(x)
    like = likelihood(x, orangers_vs_speeders.iloc[:N,0].to_numpy())
    like /= like.sum() / len(x)
    like_mean = (x * like).sum() / len(x)
    like_mode = x[np.argmax(like)]

    post = like * prior(x)
    post /= post.sum() / len(x)
    post_mean = (x * post).sum() / len(x)
    post_mode = x[np.argmax(post)]

    plt.plot(x, pri, label='Prior')
    plt.plot(x, like, label='Likelihood')
    plt.plot(x, post, label='Posterior')
    plt.legend()
    plt.show()

widgets.interact(plot, N=(1, 16), a=(1, 100), b=(1, 100))

interactive(children=(IntSlider(value=8, description='N', max=16, min=1), IntSlider(value=50, description='a',…

<function __main__.plot(N, a)>